### Extracting conifrmed cases in the US

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("./csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

def fix_year(column_name):
    if column_name.endswith("-20"):
        return column_name[:-2] + "2020"
    elif column_name.endswith("-21"):
        return column_name[:-2] + "2021"
    elif column_name.endswith("-22"):
        return column_name[:-2] + "2022"
    elif column_name.endswith("-23"):
        return column_name[:-2] + "2023"
    else:
        return column_name

# Apply the function to all column names
df = df.rename(columns=fix_year)
#print(df)

#df.to_csv("./csse_covid_19_time_series/time_series_covid19_confirmed_global.csv", index=False)


# Select rows where the Country/Region column equals "US"
df_us = df[df["Country/Region"] == "US"]

# Remove the first four columns
df_us = df_us.iloc[:, 4:]

# Transpose the dataframe
df_us = df_us.transpose()



df_us = df_us.rename(columns={df_us.columns[0]: 'Confirmed cases'})


# add a column for dates
df_us = df_us.reset_index()
df_us = df_us.rename(columns={'index': 'Date'})

df_us['Date'] = pd.to_datetime(df_us['Date'], format='%m-%d-%Y')

print(df_us)

           Date  Confirmed cases
0    2020-01-22                1
1    2020-01-23                1
2    2020-01-24                2
3    2020-01-25                2
4    2020-01-26                5
...         ...              ...
1138 2023-03-05        103646975
1139 2023-03-06        103655539
1140 2023-03-07        103690910
1141 2023-03-08        103755771
1142 2023-03-09        103802702

[1143 rows x 2 columns]


### Extracting total deaths in the US

In [3]:
# read in the second dataset
df_us_deaths = pd.read_csv('./csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

df_us_deaths = df_us_deaths.rename(columns=fix_year)

#print(df_us_deaths)

# select rows where country is US
df_us_deaths = df_us_deaths[df_us_deaths['Country/Region'] == 'US']

# drop the first four columns
df_us_deaths = df_us_deaths.iloc[:, 4:]

# transpose the dataframe
df_us_deaths = df_us_deaths.transpose()

# set the column names
df_us_deaths.columns = ['Deaths']

# add a column for dates
df_us_deaths = df_us_deaths.reset_index()
df_us_deaths = df_us_deaths.rename(columns={'index': 'Date'})

df_us_deaths = df_us_deaths.drop('Date', axis=1)
#print(df_us_deaths)


# add the Deaths column to the previous dataframe
df_us_cd = pd.concat([df_us, df_us_deaths], axis=1)
print(df_us_cd)

           Date  Confirmed cases   Deaths
0    2020-01-22                1        0
1    2020-01-23                1        0
2    2020-01-24                2        0
3    2020-01-25                2        0
4    2020-01-26                5        0
...         ...              ...      ...
1138 2023-03-05        103646975  1122134
1139 2023-03-06        103655539  1122181
1140 2023-03-07        103690910  1122516
1141 2023-03-08        103755771  1123246
1142 2023-03-09        103802702  1123836

[1143 rows x 3 columns]


### Extracting Recovered information in the US

In [4]:
# read in the second dataset
df_us_recovered = pd.read_csv('./csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

df_us_recovered = df_us_recovered.rename(columns=fix_year)

#print(df_us_deaths)

# select rows where country is US
df_us_recovered = df_us_recovered[df_us_recovered['Country/Region'] == 'US']

# drop the first four columns
df_us_recovered = df_us_recovered.iloc[:, 4:]

# transpose the dataframe
df_us_recovered = df_us_recovered.transpose()

# set the column names
df_us_recovered.columns = ['Recovered']

# add a column for dates
df_us_recovered = df_us_recovered.reset_index()
df_us_recovered = df_us_recovered.rename(columns={'index': 'Date'})

df_us_recovered = df_us_recovered.drop('Date', axis=1)
#print(df_us_deaths)


# add the Deaths column to the previous dataframe
df_us_cdr = pd.concat([df_us_cd, df_us_recovered], axis=1)
print(df_us_cdr)

           Date  Confirmed cases   Deaths  Recovered
0    2020-01-22                1        0          0
1    2020-01-23                1        0          0
2    2020-01-24                2        0          0
3    2020-01-25                2        0          0
4    2020-01-26                5        0          0
...         ...              ...      ...        ...
1138 2023-03-05        103646975  1122134          0
1139 2023-03-06        103655539  1122181          0
1140 2023-03-07        103690910  1122516          0
1141 2023-03-08        103755771  1123246          0
1142 2023-03-09        103802702  1123836          0

[1143 rows x 4 columns]


### Creating daily new cases, deaths, and active cases

In [6]:
import pandas as pd
from tabulate import tabulate

# Load the dataset
#df_us_cdr = pd.read_csv('your_dataset.csv')

# Convert the Date column to datetime
df_us_cdr['Date'] = pd.to_datetime(df_us_cdr['Date'], format='%Y-%m-%d')

# Calculate daily new cases and deaths
df_us_cdr['Daily New Cases'] = df_us_cdr['Confirmed cases'].diff().fillna(0).astype(int)
df_us_cdr['Daily Deaths'] = df_us_cdr['Deaths'].diff().fillna(0).astype(int)

# Calculate active cases
df_us_cdr['Active Cases'] = df_us_cdr['Confirmed cases'] - df_us_cdr['Deaths'] - df_us_cdr['Recovered']

# Print the updated dataframe
#print(df_us_cdr)

print(tabulate(df_us_cdr, headers='keys', tablefmt='psql'))




+------+---------------------+-------------------+----------+-------------+-------------------+----------------+----------------+
|      | Date                |   Confirmed cases |   Deaths |   Recovered |   Daily New Cases |   Daily Deaths |   Active Cases |
|------+---------------------+-------------------+----------+-------------+-------------------+----------------+----------------|
|    0 | 2020-01-22 00:00:00 |                 1 |        0 |           0 |                 0 |              0 |              1 |
|    1 | 2020-01-23 00:00:00 |                 1 |        0 |           0 |                 0 |              0 |              1 |
|    2 | 2020-01-24 00:00:00 |                 2 |        0 |           0 |                 1 |              0 |              2 |
|    3 | 2020-01-25 00:00:00 |                 2 |        0 |           0 |                 0 |              0 |              2 |
|    4 | 2020-01-26 00:00:00 |                 5 |        0 |           0 |               

### Merging S&P 500 dataset and Covid-19 dataset

In [7]:
df_sp500 = pd.read_csv('s_and_p.csv')

# Convert the Date column to datetime format
df_sp500['Date'] = pd.to_datetime(df_sp500['Date'], format='%Y-%m-%d')

# Set the Date column as the index
df_sp500 = df_sp500.set_index('Date')

# Extract the data between the dates '22-01-2020' and '09-03-2023'
df_sp500 = df_sp500.loc['2020-01-22':'2023-03-09']

print(tabulate(df_sp500, headers='keys', tablefmt='psql'))
#print(df_sp500)

df_covid_stocks = pd.merge(df_us_cdr, df_sp500, on='Date', how='inner')

#print(df_covid_stocks)

df_covid_stocks.to_csv('covid_stocks_combined.csv', index=False)

+---------------------+---------+---------+---------+---------+-------------+-------------+
| Date                |    Open |    High |     Low |   Close |   Adj Close |      Volume |
|---------------------+---------+---------+---------+---------+-------------+-------------|
| 2020-01-22 00:00:00 | 3330.02 | 3337.77 | 3320.04 | 3321.75 |     3321.75 | 3.62378e+09 |
| 2020-01-23 00:00:00 | 3315.77 | 3326.88 | 3301.87 | 3325.54 |     3325.54 | 3.76671e+09 |
| 2020-01-24 00:00:00 | 3333.1  | 3333.18 | 3281.53 | 3295.47 |     3295.47 | 3.70878e+09 |
| 2020-01-27 00:00:00 | 3247.16 | 3258.85 | 3234.5  | 3243.63 |     3243.63 | 3.83105e+09 |
| 2020-01-28 00:00:00 | 3255.35 | 3285.78 | 3253.22 | 3276.24 |     3276.24 | 3.53157e+09 |
| 2020-01-29 00:00:00 | 3289.46 | 3293.47 | 3271.89 | 3273.4  |     3273.4  | 3.60025e+09 |
| 2020-01-30 00:00:00 | 3256.45 | 3285.91 | 3242.8  | 3283.66 |     3283.66 | 3.79035e+09 |
| 2020-01-31 00:00:00 | 3282.33 | 3282.33 | 3214.68 | 3225.52 |     3225.52 | 4.